In [3]:
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
def crossCheck(cross,foldnum):
    scores=[]
    for i in range(len(cross)):

        ctrain=[]
        ctrain_y=[]
        testnum=foldnum-i-1
        testme=[]
        testme_y=[]
        
        testme=np.array(testme)
        testme_y=np.array(testme_y)
        j=0
        while j<len(cross):

            
            if j!=testnum:
                for thing in cross[j]:
                    ctrain.append(thing[0])
                    ctrain_y.append(thing[1])
               
            j+=1

# meds=[]
# with open('meds.txt') as medt:
#     meds=medt.readline().strip().split('\t')
data=pd.read_csv('diabetic_data.csv')
diag=data[['diag_1','diag_2','diag_3']]
#for thing in diag:
data=data.drop(['admission_type_id','discharge_disposition_id','admission_source_id','max_glu_serum','number_inpatient','number_emergency','number_outpatient','encounter_id','patient_nbr','weight','payer_code','A1Cresult','medical_specialty','max_glu_serum','diag_1','diag_2','diag_3'],axis=1)
data=data.drop(data[data.race=='?'].index)

data=data.drop(data[data.gender=='Unknown/Invalid'].index)
#data=data.drop(data[data.diag_3==str].index)


read=data['readmitted']
read=read.str.upper()
read=read.replace(['NO','<30','>30'],[0,1,1])

#print(read)
data=data.replace(['None','Female','No','Down'],0)
data=data.replace(['Male','Yes','Up','Steady'],1)
#data=data.replace(['[0-10)','[10-20)','[20-30)','[30-40)','[40-50)','[50-60)','[60-70)','[70-80)','[80-90)','[90-100)'],[0,1,2,3,4,5,6,7,8,9])
data=data.replace(['Down'],2)
#data=data.replace(['Other','Hispanic'],[3,4])
r=pd.get_dummies(data[['race','diabetesMed']])
change=pd.get_dummies(data['change'])
a=pd.get_dummies(data['age'])
#meddum=pd.get_dummies(data[meds])

newdata=pd.concat([data,r,a,change],axis=1)
newdata=newdata.drop(['age','race','readmitted','change','diabetesMed'],axis=1)

X=data.drop(['readmitted'],axis=1)

read=np.array(read)
x, X_test, y, y_test = train_test_split(newdata, read, test_size = 0.15, random_state = 0)
X_train,x_dev,y_train,y_dev=train_test_split(x,y,test_size=.18, random_state=0)



In [22]:
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier 


#hyperparameter grid search
mxDepth = [5,10,15] #mxDepth 
n_ests = [100] #n_estimators 
best_models = []
for n in n_ests:
    print("n_estimators =",n)
    maximized_models=[]
    for mD in mxDepth:
        results = []
        print("\tmax_depth =",mD)
        for _ in range(51):
            model = RandomForestClassifier(n_estimators=n,max_depth=mD)
            model.fit(X_train,y_train)
            pred = model.predict(x_dev)
            testy=np.array(y_dev)
            print('\t'+str(_)+" iterations",end="\r",flush=True)

            results.append((model,accuracy_score(pred,testy)))
        print('\n\tBest model score:',max(results,key=itemgetter(1))[1])
        maximized_models.append((max(results,key=itemgetter(1))[0],max(results,key=itemgetter(1))[1],mD))

    print(n,"estimators","\tbest at max_depth =",max(maximized_models,key=itemgetter(1))[2],"\nscore =",max(maximized_models,key=itemgetter(1))[1])
    print("\n")
    best_models.append((max(maximized_models,key=itemgetter(1))[0],max(maximized_models,key=itemgetter(1))[1],n))

#grid search derived best model
the_champ = max(best_models,key=itemgetter(1))[0]

#score
pred_final = the_champ.predict(X_test)
print("\nfinal test score:",accuracy_score(pred_final,np.array(y_test)))


n_estimators = 100
	max_depth = 5
	50 iterations
	Best model score: 0.558825461473
	max_depth = 10
	50 iterations
	Best model score: 0.574196938843
	max_depth = 15
	50 iterations
	Best model score: 0.577284372331
100 estimators 	best at max_depth = 15 
score = 0.577284372331



final test score: 0.574510854999



final test score: 0.569485392656
